# Object References, Mutability, and Recycling
A surprising trait of tuples is revealed: they are immutable but their values may change. This leads to a discussion of shallow and deep copies. References and function parameters are our next theme: the problem with mutable parameter defaults and the safe handling of mutable arguments passed by clients of our functions. <br>

The last sections of the chapter cover garbage collection, the del command, and how to use weak references to 'remember' objects without keeping them alive.

In [1]:
t = (1, 2, [1, 2])

In [5]:
t[0], t[1], t[2]

(1, 2, [1, 2])

In [6]:
import traceback

try:
    t[0] += 1
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-6-8908e6e22935>", line 4, in <module>
    t[0] += 1
TypeError: 'tuple' object does not support item assignment


In [7]:
import traceback

t[2].append(3)

In [8]:
t

(1, 2, [1, 2, 3])

The usual “variables as boxes” metaphor actually hinders the understanding of reference variables in OO languages. Python variables are like reference variables in Java, so it’s better to think of them as labels attached to objects.

In [9]:
class Gizmo(object):
    def __init__(self):
        print("Gizmo id: %d" % id(self))

x = Gizmo()

Gizmo id: 1701164970336


In [10]:
import traceback

try:
    y = Gizmo() * 10
except:
    traceback.print_exc()

Gizmo id: 1701164972352


Traceback (most recent call last):
  File "<ipython-input-10-b3fd75d698a5>", line 4, in <module>
    y = Gizmo() * 10
TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'


That proves that this second Gizmo was actually instantiated before the multiplication was attempted. Because variables are mere labels, nothing prevents an object from having several labels assigned to it. When that happens, you have aliasing, our next topic.

## Identity, Equality, and Aliases

In [11]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
lewis is charles

True

In [12]:
id(charles), id(lewis)

(1701164218264, 1701164218264)

In [13]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [14]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
alex == charles    

True

In [16]:
# they are equal, but they are not the same object
# and do not occupy the same space in memory.
alex is charles

False

## Choosing Between == and is
We often care about values and not identities, so == appears more frequently than is in Python code. However, if you are comparing a variable to a singleton, then it makes sense to use is. By far, the most common case is checking whether a variable is bound to None. This is the recommended way to do it:

In [19]:
x = 42
x is None

False

In [20]:
x is not None

True

In [21]:
id(None)

1554687120

The is operator is faster than ==, because it cannot be overloaded, so Python does not have to find and invoke special methods to evaluate it.

## The Relative Immutability of Tuples
Tuples, like most Python collections—lists, dicts, sets, etc.—hold references to objects. If the referenced items are mutable, they may change even if the tuple itself does not.

In [1]:
t = (1, 2, [1, 2])
t

(1, 2, [1, 2])

In [2]:
t[2].append(3)
t

(1, 2, [1, 2, 3])

## Copies Are Shallow by Default
The easiest way to copy a list (or most built-in mutable collections) is to use the builtin constructor for the type itself. For example:

In [3]:
l1 = [3, [55, 44], (7, 8, 9)]
l1

[3, [55, 44], (7, 8, 9)]

In [4]:
l2 = l1
l2 is l1

True

In [5]:
l3 = list(l1)
l3 is l1

False

In [6]:
l3 == l1

True

However, using the constructor or [:] produces a shallow copy (i.e., the outermost container is duplicated, but the copy is filled with references to the same items held by the original container).  This saves memory and causes no problems if all the items are immutable. But if there are mutable items, this may lead to unpleasant surprises.

In [7]:
l1 = [1, 2, {1, 2}]
l1

[1, 2, {1, 2}]

In [8]:
l2 = list(l1)
l2 is l1

False

In [9]:
l2[2] is l1[2]

True

In [10]:
[id(i) for i in l1] # copy is filled with references to the same items

[1529900048, 1529900080, 1893045982600]

In [11]:
[id(i) for i in l2]

[1529900048, 1529900080, 1893045982600]

In [12]:
l1, l2

([1, 2, {1, 2}], [1, 2, {1, 2}])

In [17]:
type(l1[2])

set

In [18]:
l1[2].add(3)

In [19]:
l1

[1, 2, {1, 2, 3}]

In [20]:
l2

[1, 2, {1, 2, 3}]

## Deep and Shallow Copies of Arbitrary Objects
Working with shallow copies is not always a problem, but sometimes you need to make deep copies (i.e., duplicates that do not share references of embedded objects). The copy module provides the deepcopy and copy functions that return deep and shallow copies of arbitrary objects.

In [21]:
l1 = [1, 2, {1, 2}]
l2 = list(l1)

[l1[i] is l2[i] for i in range(3)]

[True, True, True]

In [22]:
import copy
l1 = [1, 2, {1, 2}]
l2 = copy.deepcopy(l1)
[l1[i] is l2[i] for i in range(3)]

[True, True, False]

In [32]:
x = 3
y = 3
x is y

True

In [24]:
x is y

True

In [25]:
id(x), id(y)

(1529900112, 1529900112)

See ingeger objects: https://docs.python.org/3/c-api/long.html <br>
Also here on SO: https://stackoverflow.com/questions/306313/is-operator-behaves-unexpectedly-with-integers <br>
The current implementation keeps an array of integer objects for all integers between -5 and 256, when you create an int in that range you actually just get back a reference to the existing object. So it should be possible to change the value of 1. I suspect the behaviour of Python in this case is undefined.

In [30]:
x = 257
y = 257
x is y

False

In [31]:
id(x), id(y)

(1893049555856, 1893049556112)

In [34]:
x, y = 257, 257 # I'm sure there is something to this.
x is y

True

## Function Parameters as References
The only mode of parameter passing in Python is call by sharing. That is the same mode used in most OO languages, including Ruby, SmallTalk, and Java (this applies to Java reference types; primitive types use call by value). Call by sharing means that each formal parameter of the function gets a copy of each reference in the arguments. 

In [37]:
def f(a, b):
    a += b
    return a


In [38]:
x, y = 1, 2
f(x, y)

3

In [40]:
x, y # x is not changed

(1, 2)

In [41]:
x, y = [1, 2], [3, 4]

f(x, y)

[1, 2, 3, 4]

In [43]:
x, y # x has been changed

([1, 2, 3, 4], [3, 4])

## Mutable Types as Parameter Defaults: Bad Idea
